## Linear and Decision Tree Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('train.csv')
meal = pd.read_csv('meal_info.csv')
center = pd.read_csv('center_info.csv')

In [3]:
dat = data.merge(center, on = 'center_id',how='left').merge(meal,on='meal_id',how='left')

In [4]:
dat

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,1271326,145,61,1543,484.09,484.09,0,0,68,473,77,TYPE_A,4.5,Desert,Indian
456544,1062036,145,61,2304,482.09,482.09,0,0,42,473,77,TYPE_A,4.5,Desert,Indian
456545,1110849,145,61,2664,237.68,321.07,0,0,501,473,77,TYPE_A,4.5,Salad,Italian
456546,1147725,145,61,2569,243.50,313.34,0,0,729,473,77,TYPE_A,4.5,Salad,Italian


In [5]:
dat.sort_values(['center_id','meal_id','week'])

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
2370,1436842,1,10,1062,181.39,181.39,0,0,865,590,56,TYPE_B,6.3,Beverages,Italian
5273,1205013,2,10,1062,183.36,182.36,0,0,782,590,56,TYPE_B,6.3,Beverages,Italian
8175,1447751,3,10,1062,184.36,182.36,0,0,851,590,56,TYPE_B,6.3,Beverages,Italian
11064,1014968,4,10,1062,182.36,183.36,0,0,1202,590,56,TYPE_B,6.3,Beverages,Italian
13918,1003563,5,10,1062,183.39,181.39,0,0,958,590,56,TYPE_B,6.3,Beverages,Italian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345410,1255251,112,186,2956,583.03,581.03,0,0,82,649,34,TYPE_A,3.4,Fish,Continental
348601,1436164,113,186,2956,583.03,582.03,0,0,40,649,34,TYPE_A,3.4,Fish,Continental
351905,1000576,114,186,2956,583.03,581.03,0,0,27,649,34,TYPE_A,3.4,Fish,Continental
355206,1429037,115,186,2956,581.03,583.03,0,0,15,649,34,TYPE_A,3.4,Fish,Continental


In [6]:
dat =dat.drop(['id'],axis=1)

In [7]:
# dat['order_lag'] = dat['num_orders'].shift(10)
# dat['checkout_lag'] = dat['checkout_price'].shift(10)
# dat.dropna(inplace=True)

In [8]:
dat.corr()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,op_area
week,1.000000,-0.003450,0.019814,0.026581,0.028614,-0.000841,-0.008263,-0.017210,0.000405,0.004600,0.001550
center_id,-0.003450,1.000000,0.009893,0.001348,0.000604,0.013658,-0.005043,-0.053035,0.061078,-0.003426,-0.111869
meal_id,0.019814,0.009893,1.000000,0.010748,0.002605,0.013402,0.016354,0.010597,-0.003198,-0.001662,-0.001546
checkout_price,0.026581,0.001348,0.010748,1.000000,0.953389,0.004818,-0.057184,-0.282108,-0.004805,-0.003648,0.021569
base_price,0.028614,0.000604,0.002605,0.953389,1.000000,0.171173,0.057156,-0.222306,-0.002054,-0.001934,0.018031
emailer_for_promotion,-0.000841,0.013658,0.013402,0.004818,0.171173,1.000000,0.390534,0.277147,-0.005234,-0.007462,-0.019462
homepage_featured,-0.008263,-0.005043,0.016354,-0.057184,0.057156,0.390534,1.000000,0.294490,0.008640,0.003605,0.041498
num_orders,-0.017210,-0.053035,0.010597,-0.282108,-0.222306,0.277147,0.294490,1.000000,0.041596,0.029744,0.176976
city_code,0.000405,0.061078,-0.003198,-0.004805,-0.002054,-0.005234,0.008640,0.041596,1.000000,0.042686,0.131476
region_code,0.004600,-0.003426,-0.001662,-0.003648,-0.001934,-0.007462,0.003605,0.029744,0.042686,1.000000,0.023327


In [9]:
dat['city_code'] = dat['city_code'].apply(str)
dat['region_code'] = dat['region_code'].apply(str)

In [10]:
dat 

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456543,145,61,1543,484.09,484.09,0,0,68,473,77,TYPE_A,4.5,Desert,Indian
456544,145,61,2304,482.09,482.09,0,0,42,473,77,TYPE_A,4.5,Desert,Indian
456545,145,61,2664,237.68,321.07,0,0,501,473,77,TYPE_A,4.5,Salad,Italian
456546,145,61,2569,243.50,313.34,0,0,729,473,77,TYPE_A,4.5,Salad,Italian


In [11]:
# dat['area_dummy'] = dat['op_area'].apply(lambda value: 'a' if value < 3.5 else ('b' if value < 5.5 else 'c'))

In [12]:
# def meal_id(meal_id, datacol):        
#     meal_id_val_index_n = []
#     for i in datacol:
#         if i >= 100 and i <= 200:
#             meal_id_val_index_n.append(1)
#         elif i >= 201 and i <=300:
#             meal_id_val_index_n.append(2)
#         elif i >= 301 and i <=400:
#             meal_id_val_index_n.append(3)  
#         elif i >= 401 and i <=500:
#             meal_id_val_index_n.append(4)
#         elif i >= 501 and i <=600:
#             meal_id_val_index_n.append(5)          
#         else:
#             meal_id_val_index_n.append(6) 
#     return  meal_id_val_index_n

# meal_id_val_index_n = meal_id(dat.meal_id, dat.checkout_price)
# dat.meal_id = meal_id_val_index_n

In [13]:
# def center_id(center_id, datacol):        
#     center_id_val_index_n = []
#     for i in datacol:
#         if i >= 0 and i <= 2:
#             center_id_val_index_n.append(3)
#         elif i > 2 and i <=3:
#             center_id_val_index_n.append(2)
#         elif i > 3 and i <=4:
#             center_id_val_index_n.append(1)  
#         elif i >4 and i <=5:
#             center_id_val_index_n.append(5)
#         elif i > 5 and i <=6:
#             center_id_val_index_n.append(6)          
#         else:
#             center_id_val_index_n.append(4) 
#     return  center_id_val_index_n

# center_id_val_index_n = center_id(dat.center_id, dat.op_area)
# dat.center_id = center_id_val_index_n

In [40]:
dat['checkout_rolling'] = dat.groupby(['center_id','meal_id'])['checkout_price'].transform(lambda x: x.rolling(8, 1).mean())
dat['orders_rolling'] = dat.groupby('center_id')['num_orders'].transform(lambda x: x.rolling(8, 1).mean())

In [ ]:
dat['checkout_lag'] = dat.groupby('center_id')['checkout_price'].shift(4)
dat['orders_lag'] = dat.groupby('center_id')['num_orders'].shift(4)

In [17]:
dat['center_id'] = dat['center_id'].apply(str)
dat['meal_id'] = dat['meal_id'].apply(str)

In [18]:
# dat['combo']=dat['center_id']+dat['meal_id']
# dat.drop(columns=['center_id','meal_id'])

In [19]:
# dat['discount']=dat['checkout_price']-dat['base_price']

In [20]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456548 entries, 0 to 456547
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   week                   456548 non-null  int64  
 1   center_id              456548 non-null  object 
 2   meal_id                456548 non-null  object 
 3   checkout_price         456548 non-null  float64
 4   base_price             456548 non-null  float64
 5   emailer_for_promotion  456548 non-null  int64  
 6   homepage_featured      456548 non-null  int64  
 7   num_orders             456548 non-null  int64  
 8   city_code              456548 non-null  object 
 9   region_code            456548 non-null  object 
 10  center_type            456548 non-null  object 
 11  op_area                456548 non-null  float64
 12  category               456548 non-null  object 
 13  cuisine                456548 non-null  object 
 14  center_rolling         456548 non-nu

In [21]:
# dat['num_orders'] = np.log(dat['num_orders'])
# dat['order_lag'] = np.log(dat['order_lag'])

In [22]:
dat=dat.drop(columns=['city_code','center_type','category','cuisine'])

In [23]:
train_df = dat[dat['week']<=135]
test_df = dat[dat['week']>135]

In [24]:
train_df

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,region_code,op_area,center_rolling,checkout_rolling,orders_rolling
0,1,55,1885,136.83,152.29,0,0,177,56,2.0,177.000,136.830000,177.000
1,1,55,1993,136.83,135.83,0,0,270,56,2.0,223.500,136.830000,223.500
2,1,55,2539,134.86,135.86,0,0,189,56,2.0,212.000,136.173333,212.000
3,1,55,2139,339.50,437.53,0,0,54,56,2.0,172.500,187.005000,172.500
4,1,55,2631,243.50,242.50,0,0,40,56,2.0,146.000,198.304000,146.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423722,135,61,1543,482.09,484.09,0,0,55,77,4.5,65.875,464.550000,65.875
423723,135,61,2304,483.09,482.09,0,0,13,77,4.5,64.125,446.601250,64.125
423724,135,61,2664,340.53,338.53,0,0,230,77,4.5,89.375,410.832500,89.375
423725,135,61,2569,329.86,329.86,0,0,432,77,4.5,136.750,356.273750,136.750


In [25]:
train_df = train_df.drop(columns = 'week')
test_df = test_df.drop(columns = 'week')

In [26]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [27]:
train_df

,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,op_area,center_rolling,checkout_rolling,orders_rolling,center_id_10,...,meal_id_2867,meal_id_2956,region_code_23,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93
0,136.83,152.29,0,0,177,2.0,177.000,136.830000,177.000,0,...,0,0,0,0,0,1,0,0,0,0
1,136.83,135.83,0,0,270,2.0,223.500,136.830000,223.500,0,...,0,0,0,0,0,1,0,0,0,0
2,134.86,135.86,0,0,189,2.0,212.000,136.173333,212.000,0,...,0,0,0,0,0,1,0,0,0,0
3,339.50,437.53,0,0,54,2.0,172.500,187.005000,172.500,0,...,0,0,0,0,0,1,0,0,0,0
4,243.50,242.50,0,0,40,2.0,146.000,198.304000,146.000,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423722,482.09,484.09,0,0,55,4.5,65.875,464.550000,65.875,0,...,0,0,0,0,0,0,0,1,0,0
423723,483.09,482.09,0,0,13,4.5,64.125,446.601250,64.125,0,...,0,0,0,0,0,0,0,1,0,0
423724,340.53,338.53,0,0,230,4.5,89.375,410.832500,89.375,0,...,0,0,0,0,0,0,0,1,0,0
423725,329.86,329.86,0,0,432,4.5,136.750,356.273750,136.750,0,...,0,0,0,0,0,0,0,1,0,0


In [28]:
x_train = train_df.drop(columns = 'num_orders')
y_train = train_df['num_orders']

In [29]:
x_test = test_df.drop(columns = 'num_orders')
y_test = test_df['num_orders']

In [30]:
scale = StandardScaler()

scale.fit(x_train)

scaled_x_train = pd.DataFrame(scale.transform(x_train))
scaled_x_test = pd.DataFrame(scale.transform(x_test))

In [31]:
reg = linear_model.LinearRegression()
reg.fit(scaled_x_train,y_train)
y_pred1 = reg.predict(scaled_x_test)

In [32]:
print(r2_score(y_test, y_pred1)*100)

57.849106192713464


In [33]:
from sklearn.tree import DecisionTreeRegressor
DTRmodel = DecisionTreeRegressor(max_depth=21,random_state=0)
DTRmodel.fit(scaled_x_train,y_train)
y_pred2 = DTRmodel.predict(scaled_x_test)

In [34]:
print(r2_score(y_test, y_pred2)*100)

77.76407519168


In [35]:
y_pred = (y_pred1+y_pred2)/2

In [36]:
print(r2_score(y_test, y_pred)*100)


77.6362167311358


In [37]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

17109.202277221666

In [38]:
pip install anai-opensource

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [39]:
# import anai
# ai = anai.run(df=train_df, target='num_orders',predictor=['rfr'],cv_folds=2,
#               task="regression",suppress_task_detection=True)

## LSTM

In [ ]:
pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM